In [1]:
import pandas as pd
import numpy as np

In [2]:
accidents = pd.read_csv('../data/2019_clusters.csv')

In [3]:
accidents.head()

,x,y,accident_id,year,address,severity,accident_type,severity_numeric,borough_geo,timestamp,date,hour,month_name,month,day,cluster_id
0,-74.134363,4.579297,10472514,2019,KR 26-CL 52B S 2,Injury,Crash,8,TUNJUELITO,2019-01-11 14:20:00+00:00,2019-01-11,14,Jan,1,Fri,-1
1,-74.048819,4.665144,10472553,2019,CL 84-KR 8 2,Material damage,Crash,1,CHAPINERO,2019-01-11 11:30:00+00:00,2019-01-11,11,Jan,1,Fri,-1
2,-74.173301,4.615589,10472522,2019,KR 80-CL 53A S 2,Injury,Crash,8,KENNEDY,2019-01-11 23:20:00+00:00,2019-01-11,23,Jan,1,Fri,0
3,-74.025068,4.725288,10472569,2019,KR 7-CL 150 2,Material damage,Crash,1,USAQUEN,2019-01-11 06:30:00+00:00,2019-01-11,6,Jan,1,Fri,-1
4,-74.080770,4.670083,10472515,2019,CL 68-KR 58 2,Injury,Crash,8,BARRIOS UNIDOS,2019-01-10 07:20:00+00:00,2019-01-10,7,Jan,1,Thu,-1


In [4]:
df_positive = accidents[['x','y','date','hour', 'borough_geo', 'address', 'cluster_id']].copy()

In [5]:
df_positive.date = pd.to_datetime(df_positive.date)

In [6]:
df_positive['day_of_year'] = df_positive.date.dt.dayofyear

In [7]:
df_positive['x'] = round(df_positive['x'], 8)
df_positive['y'] = round(df_positive['y'], 8)

In [8]:
df_positive['coords'] = df_positive[['x', 'y']].apply(lambda x: ';'.join((str(x['x']), str(x['y']))), axis = 1)

In [9]:
df_positive = df_positive[['x','y','hour', 'day_of_year','borough_geo', 'coords', 'address', 'cluster_id']].copy()

In [10]:
negatives = pd.read_csv("../data/dataset_2019_with_negatives.csv")

In [20]:
negatives.head()

,x,y,timestamp,date,year,month,day,day_of_year,hour,address,...,windBearing,cloudCover,uvIndex,visibility,pressure,windGust,precipType,ozone,precipAccumulation,doy
0,-74.134363,4.579297,2019-01-11 14:20:00,2019-01-11,2019,1,11,11,14,KR 26-CL 52B S 2,...,167.0,0.89,4.0,13.489,1018.0,1.66,rain,239.8,NaN,11
1,-74.139488,4.573233,2019-01-11 14:00:00,2019-01-11,2019,1,11,11,14,AV AVENIDA BOYACA-KR 24 2,...,167.0,0.89,4.0,13.489,1018.0,1.66,rain,239.8,NaN,11
2,-74.048819,4.665144,2019-01-11 11:30:00,2019-01-11,2019,1,11,11,11,CL 84-KR 8 2,...,127.0,0.80,0.0,14.019,1018.0,1.52,rain,241.0,NaN,11
3,-74.173301,4.615589,2019-01-11 23:20:00,2019-01-11,2019,1,11,11,23,KR 80-CL 53A S 2,...,10.0,0.81,0.0,6.814,1014.8,2.07,rain,237.9,NaN,11
4,-74.141503,4.610874,2019-01-11 23:00:00,2019-01-11,2019,1,11,11,23,KR 72H-CL 37D S 02,...,10.0,0.81,0.0,6.814,1014.8,2.07,rain,237.9,NaN,11


In [13]:
negatives.date = pd.to_datetime(negatives.date)
negatives['doy']  = negatives.date.dt.dayofyear

In [23]:
negatives = negatives.drop(columns=['day_of_year'])                   

In [24]:
negatives.rename(columns={'doy': 'day_of_year'}, inplace=True)

In [31]:
negatives = negatives[['x', 'y', 'timestamp', 'date', 'year', 'day_of_year', 'month', 'day', 'hour',
       'address', 'severity', 'accident_type', 'severity_numeric', 'borough',
       'cluster_id', 'sample_type', 'area_km2', 'population',
       'population_density', 'timestamp_darksky', 'summary', 'icon',
       'precipIntensity', 'precipProbability', 'temperature',
       'apparentTemperature', 'dewPoint', 'humidity', 'windSpeed',
       'windBearing', 'cloudCover', 'uvIndex', 'visibility', 'pressure',
       'windGust', 'precipType', 'ozone', 'precipAccumulation']]

In [32]:
negatives.to_csv("../data/dataset_2019_with_negatives.csv", index=None)

In [19]:
negatives[negatives.doy != negatives.day_of_year][['sample_type','date', 'day_of_year', 'doy']] 

,sample_type,date,day_of_year,doy
5,1,2019-01-11,98,11
9,1,2019-01-10,98,10
15,1,2019-01-09,11,9
16,1,2019-01-09,222,9
20,1,2019-01-10,11,10
...,...,...,...,...
66487,1,2019-12-03,187,337
66489,1,2019-12-04,341,338
66490,1,2019-12-03,177,337
66498,1,2019-12-20,177,354


In [41]:
def generate_negative_sample(row):
    num_samples = 0
    while num_samples < 5:
        randcol = np.random.choice(np.array(['hour', 'day_of_year', 'coords']))
        replacement = df_positive.sample()
        negative_sample = row
        negative_sample[randcol] = replacement[randcol].values[0]
        if randcol == 'coords':
            negative_sample['borough_geo'] = replacement['borough_geo'].values[0]
            negative_sample['address']  = replacement['address'].values[0]
            negative_sample['cluster_id']  = replacement['cluster_id'].values[0]

        ncoords = negative_sample['coords']
        nhour = negative_sample['hour']
        nday = negative_sample['day_of_year']
        
        if df_positive.loc[(df_positive['coords'] == ncoords) & (df_positive['hour'] == nhour) & (df_positive['day_of_year'] == nday)].empty:
            yield negative_sample[['hour', 'day_of_year','coords','address', 'borough_geo', 'cluster_id']].values.tolist()
            num_samples += 1
        else:
            print('found coincidence', negative_sample[['hour', 'day_of_year','coords']].values.tolist())

In [50]:
my_list = [generate_negative_sample(df_positive.iloc[x]) for x in df_positive.index]

In [51]:
%%time
from itertools import chain
neg_list = list(chain.from_iterable(my_list))

 '-74.06784898;4.59400438']
found coincidence [11, 114, '-74.16646529;4.64592276']
found coincidence [5, 130, '-74.08300758;4.67338808']
found coincidence [15, 7, '-74.04531121;4.76052285']
found coincidence [6, 14, '-74.13260046;4.68008907']
found coincidence [18, 12, '-74.07256652;4.73109895']
found coincidence [13, 202, '-74.0715996;4.71125451']
found coincidence [17, 46, '-74.15129981;4.57218179']
found coincidence [7, 49, '-74.10057645;4.60103502']
found coincidence [22, 48, '-74.08907745;4.63390439']
found coincidence [17, 72, '-74.09587207;4.6224441']
found coincidence [16, 72, '-74.13710112;4.61754212']
found coincidence [8, 220, '-74.09498628;4.66424328']
found coincidence [17, 73, '-74.14307918;4.61143039']
found coincidence [13, 298, '-74.14725628;4.63450859']
found coincidence [17, 34, '-74.09174608;4.64165433']
found coincidence [16, 36, '-74.10477628;4.66712459']
found coincidence [8, 122, '-74.09182109;4.67661916']
found coincidence [21, 226, '-74.09498628;4.66424328']
f

In [66]:
neg_df = pd.DataFrame.from_records(neg_list)

In [67]:
neg_df

,0,1,2,3,4,5
0,14,11,-74.13427637;4.61257783,AV AVENIDA PRIMERA DE MAYO-KR 69 28,KENNEDY,261
1,14,11,-74.13427637;4.61257783,AV AVENIDA PRIMERA DE MAYO-KR 69 28,KENNEDY,261
2,14,11,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310
3,14,98,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310
4,14,98,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310
...,...,...,...,...,...,...
145595,9,23,-74.12992808;4.57033507,KR 19C-CL 51 S 02,TUNJUELITO,-1
145596,22,23,-74.12992808;4.57033507,KR 19C-CL 51 S 02,TUNJUELITO,-1
145597,22,23,-74.14973538;4.68733476,KR 116-CL 22F 2,FONTIBON,-1
145598,22,23,-74.13319791;4.56459794,CL 57-KR 18A S 02,TUNJUELITO,-1


In [68]:
neg_df.columns = ['hour', 'day_of_year', 'coords', 'address', 'borough', 'cluster_id']

In [69]:
neg_df['year'] = 2019

In [70]:
neg_df.shape

(145600, 7)

In [71]:
def compose_date(years, months=1, days=1, weeks=None, hours=None, minutes=None,
                 seconds=None, milliseconds=None, microseconds=None, nanoseconds=None):
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[W]', '<m8[h]',
             '<m8[m]', '<m8[s]', '<m8[ms]', '<m8[us]', '<m8[ns]')
    vals = (years, months, days, weeks, hours, minutes, seconds,
            milliseconds, microseconds, nanoseconds)
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals)
               if v is not None)


neg_df['timestamp'] = compose_date(neg_df['year'], days=neg_df['day_of_year'], hours=neg_df['hour'], minutes=np.zeros(neg_df.shape[0]), seconds=np.zeros(neg_df.shape[0]))
neg_df['date'] = compose_date(neg_df['year'], days=neg_df['day_of_year'])


In [72]:
neg_df = neg_df.join(neg_df.coords.str.split(';',expand=True))

In [73]:
neg_df.rename(columns={0: "x", 1: "y"}, inplace=True)

In [74]:
neg_df

,hour,day_of_year,coords,address,borough,cluster_id,year,timestamp,date,x,y
0,14,11,-74.13427637;4.61257783,AV AVENIDA PRIMERA DE MAYO-KR 69 28,KENNEDY,261,2019,2019-01-11 14:00:00,2019-01-11,-74.13427637,4.61257783
1,14,11,-74.13427637;4.61257783,AV AVENIDA PRIMERA DE MAYO-KR 69 28,KENNEDY,261,2019,2019-01-11 14:00:00,2019-01-11,-74.13427637,4.61257783
2,14,11,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310,2019,2019-01-11 14:00:00,2019-01-11,-74.03494633,4.68848957
3,14,98,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310,2019,2019-04-08 14:00:00,2019-04-08,-74.03494633,4.68848957
4,14,98,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310,2019,2019-04-08 14:00:00,2019-04-08,-74.03494633,4.68848957
...,...,...,...,...,...,...,...,...,...,...,...
145595,9,23,-74.12992808;4.57033507,KR 19C-CL 51 S 02,TUNJUELITO,-1,2019,2019-01-23 09:00:00,2019-01-23,-74.12992808,4.57033507
145596,22,23,-74.12992808;4.57033507,KR 19C-CL 51 S 02,TUNJUELITO,-1,2019,2019-01-23 22:00:00,2019-01-23,-74.12992808,4.57033507
145597,22,23,-74.14973538;4.68733476,KR 116-CL 22F 2,FONTIBON,-1,2019,2019-01-23 22:00:00,2019-01-23,-74.14973538,4.68733476
145598,22,23,-74.13319791;4.56459794,CL 57-KR 18A S 02,TUNJUELITO,-1,2019,2019-01-23 22:00:00,2019-01-23,-74.13319791,4.56459794


In [75]:
neg_df['accident_id'] = ''
neg_df['severity'] = ''
neg_df['accident_type'] = ''
neg_df['severity_numeric'] = ''
neg_df['date'] = neg_df.timestamp.dt.date
neg_df['day'] = neg_df.timestamp.dt.day
neg_df['month'] = neg_df.timestamp.dt.month

In [76]:
neg_df.head()

,hour,day_of_year,coords,address,borough,cluster_id,year,timestamp,date,x,y,accident_id,severity,accident_type,severity_numeric,day,month
0,14,11,-74.13427637;4.61257783,AV AVENIDA PRIMERA DE MAYO-KR 69 28,KENNEDY,261,2019,2019-01-11 14:00:00,2019-01-11,-74.13427637,4.61257783,,,,,11,1
1,14,11,-74.13427637;4.61257783,AV AVENIDA PRIMERA DE MAYO-KR 69 28,KENNEDY,261,2019,2019-01-11 14:00:00,2019-01-11,-74.13427637,4.61257783,,,,,11,1
2,14,11,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310,2019,2019-01-11 14:00:00,2019-01-11,-74.03494633,4.68848957,,,,,11,1
3,14,98,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310,2019,2019-04-08 14:00:00,2019-04-08,-74.03494633,4.68848957,,,,,8,4
4,14,98,-74.03494633;4.68848957,KR 7-CL 109 2,USAQUEN,310,2019,2019-04-08 14:00:00,2019-04-08,-74.03494633,4.68848957,,,,,8,4


In [77]:
accidents.drop(columns=['accident_id', 'month_name'], inplace=True)

In [86]:
accidents['day_of_year'] = neg_df.timestamp.dt.dayofyear

In [87]:
accidents.timestamp = pd.to_datetime(accidents.timestamp, utc=True).dt.tz_localize(None)

In [97]:
accidents['day'] = accidents.timestamp.dt.day

In [99]:
accidents.head()

,x,y,timestamp,date,year,month,day,day_of_year,hour,address,severity,accident_type,severity_numeric,borough,cluster_id,sample_type
0,-74.134363,4.579297,2019-01-11 14:20:00,2019-01-11,2019,1,11,11,14,KR 26-CL 52B S 2,Injury,Crash,8,TUNJUELITO,-1,1
1,-74.048819,4.665144,2019-01-11 11:30:00,2019-01-11,2019,1,11,11,11,CL 84-KR 8 2,Material damage,Crash,1,CHAPINERO,-1,1
2,-74.173301,4.615589,2019-01-11 23:20:00,2019-01-11,2019,1,11,11,23,KR 80-CL 53A S 2,Injury,Crash,8,KENNEDY,0,1
3,-74.025068,4.725288,2019-01-11 06:30:00,2019-01-11,2019,1,11,98,6,KR 7-CL 150 2,Material damage,Crash,1,USAQUEN,-1,1
4,-74.080770,4.670083,2019-01-10 07:20:00,2019-01-10,2019,1,10,98,7,CL 68-KR 58 2,Injury,Crash,8,BARRIOS UNIDOS,-1,1


In [88]:
accidents = accidents[['x', 'y', 'timestamp', 'date', 'year', 'month',
       'day', 'day_of_year', 'hour', 'address', 'severity', 'accident_type',
       'severity_numeric', 'borough_geo','cluster_id']]

KeyError: "['borough_geo'] not in index"

In [89]:
accidents.rename(columns={"borough_geo": "borough"}, inplace=True)

In [90]:
accidents['sample_type'] = 1

In [91]:
neg_df = neg_df[['x', 'y', 'timestamp', 'date', 'year', 'month',
       'day', 'day_of_year', 'hour', 'address', 'severity', 'accident_type',
       'severity_numeric', 'borough','cluster_id']]

In [92]:
neg_df['sample_type'] = 0

In [100]:
dataset = pd.concat([accidents,neg_df])

In [101]:
dataset

,x,y,timestamp,date,year,month,day,day_of_year,hour,address,severity,accident_type,severity_numeric,borough,cluster_id,sample_type
0,-74.1344,4.5793,2019-01-11 14:20:00,2019-01-11,2019,1,11,11,14,KR 26-CL 52B S 2,Injury,Crash,8,TUNJUELITO,-1,1
1,-74.0488,4.66514,2019-01-11 11:30:00,2019-01-11,2019,1,11,11,11,CL 84-KR 8 2,Material damage,Crash,1,CHAPINERO,-1,1
2,-74.1733,4.61559,2019-01-11 23:20:00,2019-01-11,2019,1,11,11,23,KR 80-CL 53A S 2,Injury,Crash,8,KENNEDY,0,1
3,-74.0251,4.72529,2019-01-11 06:30:00,2019-01-11,2019,1,11,98,6,KR 7-CL 150 2,Material damage,Crash,1,USAQUEN,-1,1
4,-74.0808,4.67008,2019-01-10 07:20:00,2019-01-10,2019,1,10,98,7,CL 68-KR 58 2,Injury,Crash,8,BARRIOS UNIDOS,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145595,-74.12992808,4.57033507,2019-01-23 09:00:00,2019-01-23,2019,1,23,23,9,KR 19C-CL 51 S 02,,,,TUNJUELITO,-1,0
145596,-74.12992808,4.57033507,2019-01-23 22:00:00,2019-01-23,2019,1,23,23,22,KR 19C-CL 51 S 02,,,,TUNJUELITO,-1,0
145597,-74.14973538,4.68733476,2019-01-23 22:00:00,2019-01-23,2019,1,23,23,22,KR 116-CL 22F 2,,,,FONTIBON,-1,0
145598,-74.13319791,4.56459794,2019-01-23 22:00:00,2019-01-23,2019,1,23,23,22,CL 57-KR 18A S 02,,,,TUNJUELITO,-1,0


In [102]:
boroughs = pd.read_csv("../data/localidades.csv")

In [103]:
boroughs.borough.unique()

array(['USAQUEN', 'CHAPINERO', 'SANTA FE', 'SAN CRISTOBAL', 'USME',
       'TUNJUELITO', 'BOSA', 'KENNEDY', 'FONTIBON', 'ENGATIVA', 'SUBA',
       'BARRIOS UNIDOS', 'TEUSAQUILLO', 'LOS MARTIRES', 'ANTONIO NARIÑO',
       'PUENTE ARANDA', 'CANDELARIA', 'RAFAEL URIBE URIBE',
       'CIUDAD BOLIVAR', 'SUMAPAZ'], dtype=object)

In [104]:
boroughs.columns

Index(['id', 'borough', 'area_km2', 'population', 'population_density'], dtype='object')

In [6]:
# boroughs.replace({"Tunjuelito": 'TUNJUELITO', "Chapinero":'CHAPINERO', "Kennedy": 'KENNEDY', "Usaquén": 'USAQUEN', "BarriosUnidos":  'BARRIOS UNIDOS', "CiudadBolívar": 
# 'CIUDAD BOLIVAR', "Fontibón":  'FONTIBON', "PuenteAranda":  'PUENTE ARANDA', "Suba":  'SUBA', "Teusaquillo":
# 'TEUSAQUILLO', "SantaFe":  'SANTA FE', "Bosa":  'BOSA', "SanCristóbal":  'SAN CRISTOBAL', "Engativá":  'ENGATIVA', "RafaelUribeUribe": 
# 'RAFAEL URIBE URIBE', "AntonioNariño":  'ANTONIO NARIÑO', "LosMártires":  'LOS MARTIRES', "LaCandelaria":
# 'CANDELARIA', "Usme": 'USME', "Sumapaz":  'SUMAPAZ'}, inplace=True)

In [105]:
boroughs.drop(columns = ['id'], inplace=True)

In [106]:
dataset = dataset.merge(boroughs, how='left', left_on='borough', right_on='borough')

In [107]:
dataset.head()

,x,y,timestamp,date,year,month,day,day_of_year,hour,address,severity,accident_type,severity_numeric,borough,cluster_id,sample_type,area_km2,population,population_density
0,-74.1344,4.5793,2019-01-11 14:20:00,2019-01-11,2019,1,11,11,14,KR 26-CL 52B S 2,Injury,Crash,8,TUNJUELITO,-1,1,9.91,199430,20124.11
1,-74.0488,4.66514,2019-01-11 11:30:00,2019-01-11,2019,1,11,11,11,CL 84-KR 8 2,Material damage,Crash,1,CHAPINERO,-1,1,38.15,139701,3661.88
2,-74.1733,4.61559,2019-01-11 23:20:00,2019-01-11,2019,1,11,11,23,KR 80-CL 53A S 2,Injury,Crash,8,KENNEDY,0,1,38.59,1088443,28205.31
3,-74.0251,4.72529,2019-01-11 06:30:00,2019-01-11,2019,1,11,98,6,KR 7-CL 150 2,Material damage,Crash,1,USAQUEN,-1,1,65.31,501999,7686.40
4,-74.0808,4.67008,2019-01-10 07:20:00,2019-01-10,2019,1,10,98,7,CL 68-KR 58 2,Injury,Crash,8,BARRIOS UNIDOS,-1,1,11.90,243465,20459.24


In [108]:
weather = pd.read_csv("../data/weather_full.csv")

In [109]:
weather.columns

Index(['time', 'summary', 'icon', 'precipIntensity', 'precipProbability',
       'temperature', 'apparentTemperature', 'dewPoint', 'humidity',
       'windSpeed', 'windBearing', 'cloudCover', 'uvIndex', 'visibility',
       'location', 'pressure', 'windGust', 'precipType', 'ozone',
       'precipAccumulation'],
      dtype='object')

In [110]:
weather.head()

,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windBearing,cloudCover,uvIndex,visibility,location,pressure,windGust,precipType,ozone,precipAccumulation
0,2017-10-27 05:00:00,Clear,clear-night,0.0,0.0,7.0,6.22,6.99,1.00,1.50,320.0,0.00,0.0,8.005,BOSA,NaN,NaN,NaN,NaN,NaN
1,2017-10-27 06:00:00,Clear,clear-night,0.0,0.0,7.0,6.22,6.00,0.93,1.50,320.0,0.13,0.0,8.005,BOSA,NaN,NaN,NaN,NaN,NaN
2,2017-10-27 07:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,7.0,6.22,6.99,1.00,1.50,0.0,0.44,0.0,10.003,BOSA,NaN,NaN,NaN,NaN,NaN
3,2017-10-27 08:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,9.0,8.51,9.00,1.00,1.50,0.0,0.44,0.0,8.005,BOSA,NaN,NaN,NaN,NaN,NaN
4,2017-10-27 09:00:00,Mostly Cloudy,partly-cloudy-night,0.0,0.0,10.0,10.00,9.99,1.00,1.02,251.0,0.75,0.0,8.005,BOSA,NaN,NaN,NaN,NaN,NaN


In [111]:
dataset['timestamp_floor'] =  dataset.timestamp.dt.floor('H')

In [112]:
weather.time = pd.to_datetime(weather.time)

In [113]:
weather.location.unique()

array(['BOSA', 'ENGATIVA', 'TUNJUELITO', 'KENNEDY', 'FONTIBON',
       'CHAPINERO', 'CIUDAD BOLIVAR', 'ANTONIO NARIÑO', 'CANDELARIA',
       'PUENTE ARANDA', 'RAFAEL URIBE URIBE', 'BARRIOS UNIDOS', 'SUBA',
       'SAN CRISTOBAL', 'SUMAPAZ', 'LOS MARTIRES', 'SANTA FE',
       'TEUSAQUILLO', 'USAQUEN', 'USME'], dtype=object)

In [12]:
# weather.replace({"tunjuelito": 'TUNJUELITO', "chapinero":'CHAPINERO', "kennedy": 'KENNEDY', "usaquen": 'USAQUEN', "barrios unidos":  'BARRIOS UNIDOS', "ciudadbolivar": 
# 'CIUDAD BOLIVAR', "fontibon":  'FONTIBON', "puente aranda":  'PUENTE ARANDA', "suba":  'SUBA', "teusaquillo":
# 'TEUSAQUILLO', "santafe":  'SANTA FE', "bosa":  'BOSA', "san cristobal":  'SAN CRISTOBAL', "engativa":  'ENGATIVA', "rafael uribe": 
# 'RAFAEL URIBE URIBE', "antonio nariño":  'ANTONIO NARIÑO', "los martires":  'LOS MARTIRES', "la candelaria":
# 'CANDELARIA', "usme": 'USME', "sumapaz":  'SUMAPAZ'}, inplace=True)

In [114]:
dataset = dataset.merge(weather, left_on=['borough' , 'timestamp_floor'], right_on=['location', 'time'])

In [115]:
dataset.columns

Index(['x', 'y', 'timestamp', 'date', 'year', 'month', 'day', 'day_of_year',
       'hour', 'address', 'severity', 'accident_type', 'severity_numeric',
       'borough', 'cluster_id', 'sample_type', 'area_km2', 'population',
       'population_density', 'timestamp_floor', 'time', 'summary', 'icon',
       'precipIntensity', 'precipProbability', 'temperature',
       'apparentTemperature', 'dewPoint', 'humidity', 'windSpeed',
       'windBearing', 'cloudCover', 'uvIndex', 'visibility', 'location',
       'pressure', 'windGust', 'precipType', 'ozone', 'precipAccumulation'],
      dtype='object')

In [116]:
dataset.drop(columns = ['timestamp_floor', 'location'], inplace=True)

In [117]:
dataset.rename(columns={"time": "timestamp_darksky"}, inplace=True)

In [120]:
dataset.to_csv('../data/dataset_2019_with_negatives.csv', index=None)

In [119]:
dataset

,x,y,timestamp,date,year,month,day,day_of_year,hour,address,...,windSpeed,windBearing,cloudCover,uvIndex,visibility,pressure,windGust,precipType,ozone,precipAccumulation
0,-74.1344,4.5793,2019-01-11 14:20:00,2019-01-11,2019,1,11,11,14,KR 26-CL 52B S 2,...,1.66,167.0,0.89,4.0,13.489,1018.0,1.66,rain,239.8,NaN
1,-74.1394883,4.57323292,2019-01-11 14:00:00,2019-01-11,2019,1,11,11,14,AV AVENIDA BOYACA-KR 24 2,...,1.66,167.0,0.89,4.0,13.489,1018.0,1.66,rain,239.8,NaN
2,-74.0488,4.66514,2019-01-11 11:30:00,2019-01-11,2019,1,11,11,11,CL 84-KR 8 2,...,1.09,127.0,0.80,0.0,14.019,1018.0,1.52,rain,241.0,NaN
3,-74.1733,4.61559,2019-01-11 23:20:00,2019-01-11,2019,1,11,11,23,KR 80-CL 53A S 2,...,1.65,10.0,0.81,0.0,6.814,1014.8,2.07,rain,237.9,NaN
4,-74.14150261,4.61087396,2019-01-11 23:00:00,2019-01-11,2019,1,11,11,23,KR 72H-CL 37D S 02,...,1.65,10.0,0.81,0.0,6.814,1014.8,2.07,rain,237.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174349,-74.17421687,4.57624236,2019-04-26 16:00:00,2019-04-26,2019,4,26,116,16,KR 75-CL 75D S 02,...,1.62,154.0,0.90,7.0,14.461,1017.9,2.68,rain,246.0,NaN
174350,-74.12992808,4.57033507,2019-01-23 09:00:00,2019-01-23,2019,1,23,23,9,KR 19C-CL 51 S 02,...,2.48,127.0,0.78,0.0,16.093,1017.7,4.72,rain,239.2,NaN
174351,-74.12992808,4.57033507,2019-01-23 22:00:00,2019-01-23,2019,1,23,23,22,KR 19C-CL 51 S 02,...,2.72,129.0,0.77,1.0,16.093,1013.5,4.14,rain,241.0,NaN
174352,-74.13319791,4.56459794,2019-01-23 22:00:00,2019-01-23,2019,1,23,23,22,CL 57-KR 18A S 02,...,2.72,129.0,0.77,1.0,16.093,1013.5,4.14,rain,241.0,NaN
